# Load libraries

In [1]:
import pandas as pd
#from sklearn.cross_validation import KFold,StratifiedKFold
#import datetime
import numpy as np
#from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
#from sklearn import metrics
#from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn import metrics
from sklearn.cross_validation import StratifiedKFold
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR

#import seaborn as sns
#import pickle
#from sklearn import metrics
#import sys

import matplotlib.pyplot as plt

/home/godzy/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
plt.style.use('ggplot')

%matplotlib inline

#  
# Load data

In [3]:
df = pd.read_csv('../../data/cs-training.csv')

In [4]:
df.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [5]:
df.rename(columns={df.columns[0]:'id'}, inplace=True)

In [6]:
df.head()

,id,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [7]:
len(df)

150000

In [8]:
df.mean(axis=0)

id                                      75000.500000
SeriousDlqin2yrs                            0.066840
RevolvingUtilizationOfUnsecuredLines        6.048438
age                                        52.295207
NumberOfTime30-59DaysPastDueNotWorse        0.421033
DebtRatio                                 353.005076
MonthlyIncome                            6670.221237
NumberOfOpenCreditLinesAndLoans             8.452760
NumberOfTimes90DaysLate                     0.265973
NumberRealEstateLoansOrLines                1.018240
NumberOfTime60-89DaysPastDueNotWorse        0.240387
NumberOfDependents                          0.757222
dtype: float64

In [9]:
df.rename(columns={'SeriousDlqin2yrs':'target'}, inplace=True)

features = [c for c in df.columns if c not in ['id','target']]

#  
# Replacing NANs

In [10]:
for c in df.columns:
    nan_count = len(df[df[c].isnull()])
    if nan_count>0:
        print(c, nan_count, 1.0*nan_count/len(df))

MonthlyIncome 29731 0.19820666666666667
NumberOfDependents 3924 0.02616


In [11]:
print(min(df['MonthlyIncome']))
print(min(df['NumberOfDependents']))

0.0
0.0


In [12]:
df = df.fillna(-1)

#  
# Functions

In [19]:
def getResultsWithCV(dataframe, 
                     clf, 
                     target_column, 
                     features, 
                     folds=5, 
                     balance_classes=False, 
                     balancing_factor=3):

    kf = StratifiedKFold(y=dataframe[target_column], 
                           n_folds=folds, 
                           shuffle=True,
                           random_state=42)
    
    if list(dataframe.index.values) != list(range(len(dataframe))):
        print("ERROR WITH INDEX")
        return 0
  
    AUCs = [] 
    for train_index, test_index in kf:
         
        # Train the model
        dataframe_train = dataframe.iloc[train_index]
        if balance_classes==True:
            class_1_samples = len(dataframe_train[dataframe_train[target_column]==1])
            dataframe_train = pd.concat([dataframe_train[dataframe_train[target_column]==0].sample(n=class_1_samples*balancing_factor),
                                         dataframe_train[dataframe_train[target_column]==1]])     
        X_train = dataframe_train[features].values
        y_train = dataframe_train[target_column].values     
        clf.fit(X_train, y_train)
               
        # Calculate the predictions
        dataframe_test = dataframe.iloc[test_index]
        X_test = dataframe_test[features].values
        y_test = dataframe_test[target_column].values   
        
        
        predictions_proba = clf.predict_proba(X_test)[:,1]
        dataframe.loc[test_index, 'prediction_proba'] = predictions_proba
      
        fold_auc = metrics.roc_auc_score(y_test,predictions_proba)
        AUCs.append(fold_auc)

        print( round(fold_auc,4))
    print( np.mean(AUCs), np.std(AUCs))

#  
# Run the model

In [14]:
rf = RandomForestClassifier(n_estimators=100)

In [16]:
features = [c for c in df.columns if c not in ['id','target']]

In [17]:
getResultsWithCV(dataframe=df, 
                 clf=rf, 
                 target_column='target', 
                 features=features, 
                 folds=5, 
                 balance_classes=False, 
                 balancing_factor=-1)

0.834
0.8411
0.8408
0.838
0.8405
(0.83889999999999998, 0.00266)


In [20]:
for balancing_factor in [1,2,5,10]:
    print("Balancing factor: "+str(balancing_factor))
    getResultsWithCV(dataframe=df, 
                 clf=rf, 
                 target_column='target', 
                 features=features, 
                 folds=5, 
                 balance_classes=True, 
                 balancing_factor=balancing_factor)

Balancing factor: 1
0.8452
0.8562
0.8539
0.8504
0.8524
0.851611969901 0.00372126517529
Balancing factor: 2
0.845
0.8547
0.854
0.8488
0.8542
0.851354124315 0.00380425264735
Balancing factor: 5
0.842
0.8549
0.8524
0.8486
0.8509
0.849760872926 0.00438273660367
Balancing factor: 10
0.8395
0.8465
0.8455
0.8417
0.8451
0.843638300354 0.00264344442622


#  
#  
# Generate the solution

In [44]:
df_toPredict = pd.read_csv('../../data/cs-test.csv')
df_toPredict = df_toPredict.fillna(-1)

In [23]:
df.groupby('target').count()[['id']]

,id
target,
0,139974
1,10026


### Generate 10 datasets using sampling where class are balanced  and use the mean of the 5 predictions

In [47]:
df_target1 = df[df['target']==1].copy().reset_index(drop=True)
df_target0 = df[df['target']==0].copy()

In [48]:
#dfs = []
for i in range(10):
    df_balanced = pd.concat([df_target1,
                             df_target0.sample(n=len(df_target1))]).reset_index(drop=True)
    rf.fit(df_balanced[features],
           df_balanced['target'])
    df_toPredict['Probability'+str(i)] = rf.predict_proba(df_toPredict[features])[:,1]
    

In [50]:
df_toPredict.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,...,Probability0,Probability1,Probability2,Probability3,Probability4,Probability5,Probability6,Probability7,Probability8,Probability9
0,1,-1.0,0.885519,43,0,0.177513,5700.0,4,0,0,...,0.48,0.28,0.48,0.67,0.38,0.52,0.39,0.39,0.29,0.37
1,2,-1.0,0.463295,57,0,0.527237,9141.0,15,0,4,...,0.28,0.35,0.40,0.45,0.31,0.33,0.33,0.34,0.32,0.45
2,3,-1.0,0.043275,59,0,0.687648,5083.0,12,0,1,...,0.12,0.19,0.12,0.16,0.24,0.19,0.11,0.17,0.20,0.18
3,4,-1.0,0.280308,38,1,0.925961,3200.0,7,0,2,...,0.31,0.43,0.25,0.25,0.55,0.52,0.41,0.39,0.35,0.36
4,5,-1.0,1.000000,27,0,0.019917,3865.0,4,0,0,...,0.67,0.62,0.66,0.74,0.76,0.59,0.75,0.63,0.66,0.71


In [53]:
df_toPredict['Probability'] = df_toPredict[['Probability'+str(i) for i in range(10)]].mean(axis=1)

In [55]:
df_toPredict.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,...,Probability1,Probability2,Probability3,Probability4,Probability5,Probability6,Probability7,Probability8,Probability9,Probability
0,1,-1.0,0.885519,43,0,0.177513,5700.0,4,0,0,...,0.28,0.48,0.67,0.38,0.52,0.39,0.39,0.29,0.37,0.425
1,2,-1.0,0.463295,57,0,0.527237,9141.0,15,0,4,...,0.35,0.40,0.45,0.31,0.33,0.33,0.34,0.32,0.45,0.356
2,3,-1.0,0.043275,59,0,0.687648,5083.0,12,0,1,...,0.19,0.12,0.16,0.24,0.19,0.11,0.17,0.20,0.18,0.168
3,4,-1.0,0.280308,38,1,0.925961,3200.0,7,0,2,...,0.43,0.25,0.25,0.55,0.52,0.41,0.39,0.35,0.36,0.382
4,5,-1.0,1.000000,27,0,0.019917,3865.0,4,0,0,...,0.62,0.66,0.74,0.76,0.59,0.75,0.63,0.66,0.71,0.679


In [56]:
df_toPredict.rename(inplace=True, columns={df_toPredict.columns[0]:'Id'})

In [57]:
df_toPredict[['Id','Probability']].to_csv('predictions.csv',index=False)